This code describes a simple strategy for the goose

Each action (North, South, East and West) is given a certain amount of points

1) An action that gets you closer to food brings points
     The closest the food, the more points, with a maximum of 20

2) An action that makes you do a 180° flip brings you -100 points

3) An action that makes you collide with another goose brings you -100 points

The best action is chosen

It does not use AI, Q learning or other stuffs

Please upvote if you like it :-)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# dictionary that contains the different players
# key = player index, value = Player()
dico_players = {}
# The agent
def agent(obs_dict, config_dict):
    

    import numpy as np # linear algebra
    import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
    from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, translate
    
    class Params:
        verbose = False
        
    class Game_action():
        # This class contains an action name (North, South, East or West)
        # and the associated points
        #
        # Attributes :
        #   action (str) : action name 
        #   points (int) : number of points for the action
        
        def __init__(self, action):
            self.action = action
            self.points = 0
        
        def add_points(self,value):
            self.points += value
            
        def get_points(self):
            return self.points
        def get_action(self):
            return self.action
            
    class Player():
        # This class contains a player
        # A player has 4 Game_actions and 1 last action
        # 
        # Attributes :
        #   index (int) : player index
        #   last_action(str) : last action played
        #   north, south, east, west (Game_action) : one action with its points
        
        def __init__(self, index):
            self.index = index
            self.last_action = None
        
        def set_north(self,action):
            self.north = action
        def set_south(self,action):
            self.south = action
        def set_east(self,action):
            self.east = action
        def set_west(self,action):
            self.west = action
            
        def set_last(self,action):
            self.last_action = action
            
        def get_north(self):
            return self.north
        def get_south(self):
            return self.south
        def get_east(self):
            return self.east
        def get_west(self):
            return self.west
        
        def get_last(self):
            return self.last_action
        
        def best_action(self):
            # returns the best actions (with the most points)
            if self.north.get_points() >= self.south.get_points() and self.north.get_points() >= self.east.get_points() and self.north.get_points() >= self.west.get_points():
                best_action = self.north
            elif self.south.get_points() > self.north.get_points() and self.south.get_points() >= self.east.get_points() and self.south.get_points() >= self.west.get_points():
                best_action = self.south
            elif self.east.get_points() > self.north.get_points() and self.east.get_points() > self.south.get_points() and self.east.get_points() >= self.west.get_points():
                best_action = self.east
            else:
                best_action = self.west
            #print("-----------------")
            #print(self.north.get_points())
            #print(self.south.get_points())
            #print(self.east.get_points())
            #print(self.west.get_points())
            return best_action.get_action()
        
    def distance(row1,col1,row2,col2,n_row,n_col):
        # gives the Manhattan distance  
        # from (row1, col1) to (row2, col2)
        delta_row = min(abs(row2-row1),n_row-abs(row2-row1))
        delta_col = min(abs(col2-col1),n_col-abs(col2-col1))
        return delta_row + delta_col
    
    def best_dir(row1,col1,row2,col2,n_row,n_col,index):
        # Adds points if an actions sends you 
        # from (row1, col1) to (row2, col2)
        delta_row = row2-row1
        delta_col = col2-col1
        dist = distance(row1,col1,row2,col2,n_row,n_col)
        max_dist = (n_row+n_col)//2
        # gives more points to actions that leads to closer food
        points = 20*(max_dist-dist+1)//max_dist
        actions = []
        if delta_row > 0:
            if delta_row <= n_row//2:
                dico_players[index].get_south().add_points(points)
            else:
                dico_players[index].get_north().add_points(points)
        elif delta_row < 0:
            if abs(delta_row) <= n_row//2:
                dico_players[index].get_north().add_points(points)
            else:
                dico_players[index].get_south().add_points(points)
        if delta_col > 0:
            if delta_col <= n_col//2:
                dico_players[index].get_east().add_points(points)
            else:
                dico_players[index].get_west().add_points(points)
        elif delta_col< 0:
            if abs(delta_col) <= n_col//2:
                dico_players[index].get_west().add_points(points)
            else:
                dico_players[index].get_east().add_points(points)
    
    def no_turn_back(index):
        # Penalize moves where the goose goes 180° backwards
        if dico_players[index].get_last() == Action.SOUTH.name:
            dico_players[index].get_north().add_points(-100)
        elif dico_players[index].get_last() == Action.NORTH.name:
            dico_players[index].get_south().add_points(-100)
        elif dico_players[index].get_last() == Action.WEST.name:
            dico_players[index].get_east().add_points(-100)
        else:
            dico_players[index].get_west().add_points(-100)

    def no_collision(observation,player_index,player_head,n_col,n_row):
        # Penalize moves where the goose hits another goose or itself
        
        # list of places with the other geese
        list_other = []
        for ind, goose in enumerate(observation.geese):
            #if ind != player_index:
            for place in goose:
                list_other.append(place)

        if translate(player_head,Action.NORTH,n_col,n_row) in list_other:
            dico_players[player_index].get_north().add_points(-100)
        if translate(player_head,Action.SOUTH,n_col,n_row) in list_other:
            dico_players[player_index].get_south().add_points(-100)
        if translate(player_head,Action.EAST,n_col,n_row) in list_other:
            dico_players[player_index].get_east().add_points(-100)
        if translate(player_head,Action.WEST,n_col,n_row) in list_other:
            dico_players[player_index].get_west().add_points(-100)
            
    # Observations and setup
    observation = Observation(obs_dict)
    #print(observation)
    configuration = Configuration(config_dict)
    #print(configuration)
    n_col = configuration.columns
    n_row = configuration.rows
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    
    # Initialization of dico_players
    if observation.step == 0:
        dico_players[observation.index] = Player(player_index)
    # At each step, create new Game_action for the player (with points = 0)
    dico_players[player_index].set_north(Game_action(Action.NORTH.name))
    dico_players[player_index].set_south(Game_action(Action.SOUTH.name))
    dico_players[player_index].set_east(Game_action(Action.EAST.name))
    dico_players[player_index].set_west(Game_action(Action.WEST.name))
    
    # reward good actions and penalize bad actions
    for food in observation.food:
        food_row, food_column = row_col(food, configuration.columns)
        best_dir(player_row,player_column,food_row,food_column,n_row,n_col,player_index)
    no_turn_back(player_index)
    no_collision(observation,player_index,player_head,n_col,n_row)
    
    # select the best action
    action = dico_players[player_index].best_action()
    
    # update the last action
    dico_players[observation.index].set_last(action)
    
    if Params.verbose == True:
        print(action)
    
    return action

In [ ]:
# Evaluate agent

from kaggle_environments import make, evaluate

# Create the game environment
# Set debug=True to see the errors if your agent refuses to run
env = make("hungry_geese", debug=True)

# Two random agents play one game round
a = env.run([agent,agent,agent,agent])


# Show the game
env.render(mode="ipython", width=750, height=600)



In [ ]:
from kaggle_environments import  evaluate
def get_win_percentages(agent1, agent2, n_rounds=20):
    # Use default Connect Four setup
    #config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("hungry_geese", [agent1, agent2], num_episodes=n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("hungry_geese", [agent2, agent1], num_episodes= n_rounds-n_rounds//2)]
    p1_wins = [True for [a,b] in outcomes if a>b]
    p2_wins = [True for [a,b] in outcomes if b>a]
    print("Agent 1 Win Percentage:", np.round(len(p1_wins)/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(len(p2_wins)/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))
    print(outcomes)

In [ ]:
# Win percentage
get_win_percentages(agent1=agent, agent2="greedy")

In [ ]:
#help(env)
#env.observation
env.steps

In [ ]:
#print(env.name)
#help(env)
#help(Observation)
#help(Configuration)
#help(Action)
#help(row_col)

In [ ]:
# Write agent to file

import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "w") as f:
        f.write("dico_players = {}\n")
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(agent, "submission.py")